### Danger Level Forecasting Preliminary Models

Testing out some of the classification models

#### Import Tools

In [32]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# sklearn 
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, ExtraTreesClassifier
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import StandardScaler

#### Import Data

In [33]:
avi = pd.read_csv('SnowWeatherClean.csv')
avi.head()

,Unnamed: 0,avi_danger,avg_wind,temp_max_swing,temp_max_swing_from_avg,year,month,day,temp_max,temp_min,...,northwest_wind_hours,prevailing_wind_E,prevailing_wind_N,prevailing_wind_NE,prevailing_wind_NW,prevailing_wind_S,prevailing_wind_SE,prevailing_wind_SW,prevailing_wind_W,next_day_avi_danger
0,0,2.0,10.74,0.0,0.0,2010.0,12.0,18.0,12,4,...,12,0,0,0,1,0,0,0,0,1.0
1,1,1.0,9.40,3.0,0.0,2010.0,12.0,19.0,15,3,...,12,0,0,0,1,0,0,0,0,1.0
2,2,1.0,20.58,0.0,0.0,2010.0,12.0,20.0,15,5,...,0,0,1,0,0,0,0,0,0,3.0
3,3,3.0,35.12,3.0,0.0,2010.0,12.0,21.0,18,10,...,0,0,1,0,0,0,0,0,0,2.0
4,4,2.0,33.78,-3.0,0.0,2010.0,12.0,22.0,15,7,...,1,0,1,0,0,0,0,0,0,3.0


#### filter data

In [34]:
avi = avi.drop(['Unnamed: 0'], axis=1)

In [35]:
avi = avi[avi['avi_danger'].notnull()]
avi.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1253 entries, 0 to 1252
Data columns (total 25 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   avi_danger               1253 non-null   float64
 1   avg_wind                 1253 non-null   float64
 2   temp_max_swing           1253 non-null   float64
 3   temp_max_swing_from_avg  1253 non-null   float64
 4   year                     1253 non-null   float64
 5   month                    1253 non-null   float64
 6   day                      1253 non-null   float64
 7   temp_max                 1253 non-null   int64  
 8   temp_min                 1253 non-null   int64  
 9   water_equivalent         1253 non-null   float64
 10  snow_fall                1253 non-null   float64
 11  snow_depth_6am           1253 non-null   float64
 12  wind_speed_sum           1253 non-null   int64  
 13  sunshine_percent         1253 non-null   int64  
 14  west_wind_hours         

#### Fill Remaing NA's

In [37]:
avi = avi.fillna(0)
avi = avi[avi.avi_danger != 5]
avi = avi[avi.next_day_avi_danger != 5]
avi.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1249 entries, 0 to 1252
Data columns (total 25 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   avi_danger               1249 non-null   float64
 1   avg_wind                 1249 non-null   float64
 2   temp_max_swing           1249 non-null   float64
 3   temp_max_swing_from_avg  1249 non-null   float64
 4   year                     1249 non-null   float64
 5   month                    1249 non-null   float64
 6   day                      1249 non-null   float64
 7   temp_max                 1249 non-null   int64  
 8   temp_min                 1249 non-null   int64  
 9   water_equivalent         1249 non-null   float64
 10  snow_fall                1249 non-null   float64
 11  snow_depth_6am           1249 non-null   float64
 12  wind_speed_sum           1249 non-null   int64  
 13  sunshine_percent         1249 non-null   int64  
 14  west_wind_hours         

#### Splitting Dataset

In [44]:
x = avi.iloc[:, 0:avi.shape[1]-2]
y = avi.iloc[:, avi.shape[1]-1]

print(x.shape)
print(y.shape)


(1249, 23)
(1249,)


#### Standardizing

In [45]:
#standardizer 
def standardize(X_train, X_test):
    scaler = StandardScaler()
    # Fitting and transforming training data
    scaler.fit(X_train)
    X_train = scaler.transform(X_train)
    # Tranforming testing data based on traning fit (prevent data leakage)
    X_test = scaler.transform(X_test)
    return X_train, X_test

In [46]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=42)

# Scaling
x_train, x_test = standardize(x_train, x_test)

In [47]:
print(y)

(unique, counts) = np.unique(y, return_counts=True)
frequencies = np.asarray((unique, counts)).T

print(frequencies)


0       1.0
1       1.0
2       3.0
3       2.0
4       3.0
       ... 
1248    3.0
1249    3.0
1250    3.0
1251    2.0
1252    1.0
Name: next_day_avi_danger, Length: 1249, dtype: float64
[[  1. 371.]
 [  2. 473.]
 [  3. 337.]
 [  4.  68.]]


### Descision Trees 

In [48]:
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
              }


ds = DecisionTreeClassifier()
ds_cv = RandomizedSearchCV(estimator=ds, param_distributions=random_grid, n_iter=100, scoring='f1_weighted')
ds_cv.fit(x_train, y_train)
y_pred_ds = ds_cv.predict(x_test)
print(accuracy_score(y_test, y_pred_ds))
print(cross_val_score(ds, x_train, y_train, cv=3))

0.5591054313099042
[0.51282051 0.49358974 0.49679487]


In [49]:
print(confusion_matrix(y_test, y_pred_ds))

[[72 13 10  0]
 [13 65 29  1]
 [ 9 38 30 11]
 [ 5  6  3  8]]


In [50]:

print(classification_report(y_test, y_pred_ds))

              precision    recall  f1-score   support

         1.0       0.73      0.76      0.74        95
         2.0       0.53      0.60      0.57       108
         3.0       0.42      0.34      0.37        88
         4.0       0.40      0.36      0.38        22

    accuracy                           0.56       313
   macro avg       0.52      0.52      0.52       313
weighted avg       0.55      0.56      0.55       313



### Random Forest

In [51]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}


rf = RandomForestClassifier(class_weight = "balanced")
rf_cv = RandomizedSearchCV(estimator=rf, param_distributions=random_grid, n_iter=5, scoring='f1_weighted')
rf_cv.fit(x_train, y_train)
y_pred_rf = rf_cv.predict(x_test)
print(accuracy_score(y_test, y_pred_rf))
print(cross_val_score(rf, x_train, y_train, cv=3))

0.6070287539936102
[0.61217949 0.63141026 0.59935897]


In [52]:
print(confusion_matrix(y_test, y_pred_rf))

[[75 15  5  0]
 [12 72 22  2]
 [10 38 40  0]
 [ 2  7 10  3]]


In [53]:
print(classification_report(y_test, y_pred_rf))

              precision    recall  f1-score   support

         1.0       0.76      0.79      0.77        95
         2.0       0.55      0.67      0.60       108
         3.0       0.52      0.45      0.48        88
         4.0       0.60      0.14      0.22        22

    accuracy                           0.61       313
   macro avg       0.61      0.51      0.52       313
weighted avg       0.61      0.61      0.59       313



### Extra Forest

In [54]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

et = ExtraTreesClassifier(class_weight = "balanced")
et_cv = RandomizedSearchCV(estimator=et, param_distributions=random_grid, n_iter=5, scoring='f1_weighted')
et_cv.fit(x_train, y_train)
y_pred_et = et_cv.predict(x_test)
print(accuracy_score(y_test, y_pred_et))
print(cross_val_score(rf, x_train, y_train, cv=3))

0.610223642172524
[0.61217949 0.60576923 0.59615385]


In [55]:
print(confusion_matrix(y_test, y_pred_et))

[[77 11  7  0]
 [13 67 26  2]
 [12 33 42  1]
 [ 3  6  8  5]]


In [56]:
print(classification_report(y_test, y_pred_et))

              precision    recall  f1-score   support

         1.0       0.73      0.81      0.77        95
         2.0       0.57      0.62      0.60       108
         3.0       0.51      0.48      0.49        88
         4.0       0.62      0.23      0.33        22

    accuracy                           0.61       313
   macro avg       0.61      0.53      0.55       313
weighted avg       0.61      0.61      0.60       313

